## Detectar Faces em uma imagem - Versão Jupyter Notebook

In [2]:
# importar bibliotecas
import numpy as np
import argparse
import cv2

In [3]:
# parâmetros para execução
image_file = 'images/johnson.jpeg'
prototxt = 'deploy.prototxt.txt'
model = 'res10_300x300_ssd_iter_140000.caffemodel'
minimum_confidence = 0.7

In [4]:
# carregar o modelo serializado do HD
print("[INFO] carregando modelo...")
net = cv2.dnn.readNetFromCaffe(prototxt, model)

[INFO] carregando modelo...


In [5]:
# carregar a imagem de entrada e construir um blob pra esta imagem
# através do resize para um tamanho fixo de 300x300 pixels e então normalizar
# COMMENT: Por que estes valores para normalização? https://www.pyimagesearch.com/2017/11/06/deep-learning-opencvs-blobfromimage-works/
image = cv2.imread(image_file)
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
    (300,300), (104.0, 117.0, 123.0))

In [6]:
# passe o blob pela rede neural, obtenha as detecções e
# predições
print("[INFO] Calculando detecções de objetos")
net.setInput(blob)
detections = net.forward()

[INFO] Calculando detecções de objetos


In [7]:
print(detections.shape)

(1, 1, 200, 7)


O shape retornado pela rede tem 200 possíveis rostos.
Pra cada rosto, um array com 7 elementos:
- 0 - Id Batch
- 1 - Id Classe
- 2 - Percentual de Confiança
- 3 - Esquerda (Bounding Box)
- 4 - Topo (Bounding Box)
- 5 - Direita (Bounding Box)
- 6 - Base (Bounding Box)

In [7]:
# loop sob as detecções
for i in range(0, detections.shape[2]):
    # extrair a confiança (probabilidade) associada com a
    # predição
    confidence = detections[0, 0, i, 2]

    # Eliminar detecções fracas, garantindo que a confiança é
    # maior que a confiança esperada (argumento)
    if confidence > minimum_confidence:
        # calcular as coordenadas (X, y) do bounding box para o
        # objeto
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        # desenhar o bounding box do rosto com a probabilidade
        # associada
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(image, (startX, startY), (endX, endY),
            (0, 0, 255), 2)
        cv2.putText(image, text, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

In [8]:
# mostrar a imagem de saída
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()